# Segmenting and Clustering Neighborhoods in Toronto

In [84]:
import pandas as pd
import numpy as np
import requests
df = pd.read_csv('data.csv')
geo = pd.read_csv('Geospatial_Coordinates.csv')

In [87]:
N = df.shape[0]
df = df.assign(Lon = np.zeros(N))
df = df.assign(Lat = np.zeros(N))
for n in range(0,N):
    df.loc[n,'Lat'] = geo.loc[n,'Latitude']
    df.loc[n,'Lon'] = geo.loc[n,'Longitude']
df.head()

,Postcode,Borough,Neighborhood,Lon,Lat
0,M1B,Scarborough,"Malvern, Rouge",-79.194353,43.806686
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",-79.160497,43.784535
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",-79.188711,43.763573
3,M1G,Scarborough,Woburn,-79.216917,43.770992
4,M1H,Scarborough,Cedarbrae,-79.239476,43.773136


In [88]:
# create map of Toronto using latitude and longitude values
import folium
latitude = 43.6487
longitude = -79.38544
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Lat'], df['Lon'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)
    
map_newyork

In [89]:
CLIENT_ID = 'QRQV2F25X345H0Y244BQTDNNQZT3SNBA02VYPDIWIIXGYMPW' # your Foursquare ID
CLIENT_SECRET = '2500R2BVXUKLIZWX5ONDVS5I1GAHSYPLNIT0U5COA1J2RS03' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 50
radius = 500

In [90]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [91]:

torronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Lat'],
                                   longitudes=df['Lon']
                                  )


Malvern, Rouge
Highland Creek, Port Union, Rouge Hill
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South, Flemingdon Park
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
River

In [94]:
print(torronto_venues.shape)
torronto_venues.head()

(1690, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [95]:
torronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",10,10,10,10,10,10


In [96]:
# one hot encoding
torronto_onehot = pd.get_dummies(torronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torronto_onehot['Neighborhood'] = torronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torronto_onehot.columns[-1]] + list(torronto_onehot.columns[:-1])
torronto_onehot = torronto_onehot[fixed_columns]

torronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
torronto_onehot.shape

(1690, 251)

In [98]:
torronto_grouped = torronto_onehot.groupby('Neighborhood').mean().reset_index()
torronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.020000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00000,0.000000
1,Agincourt,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00000,0.000000
4,"Alderwood, Long Branch",0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.058824,0.000000,0.000000,0.00,0.00,0.00000,0.000000
6,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.000000,0.0,0.00,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00000,0.000000
7,Bayview Village,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00000,0.000000
8,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00000,0.000000
9,Berczy Park,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00000,0.000000


In [99]:
torronto_grouped.shape

(100, 251)

In [100]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [101]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torronto_grouped['Neighborhood']

for ind in np.arange(torronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Coffee Shop,Café,Gastropub,Hotel,Pizza Place,American Restaurant,Asian Restaurant,Breakfast Spot,Restaurant
1,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Sandwich Place,Coffee Shop,Airport Service,Curling Ice
4,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Coffee Shop,Gym,Garden


In [102]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 10

torronto_grouped_clustering = torronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
kmeans.labels_

array([3, 3, 7, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3,
       3, 3, 1, 3, 9, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 1, 6, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 6, 1, 3, 3, 7, 3, 1, 3, 3, 3,
       3, 2, 3, 3, 1, 3, 8, 1, 3, 3, 1, 3, 3, 3, 3, 7, 3, 3, 0, 3, 5, 3,
       3, 3, 1, 3, 1, 1, 7, 3, 1, 1, 3, 7])

In [103]:
torronto_merged = df
torronto_merged = torronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

torronto_merged = torronto_merged.dropna(how = 'any')
torronto_merged['Cluster Labels'] = torronto_merged['Cluster Labels'].astype(int)
torronto_merged

,Postcode,Borough,Neighborhood,Lon,Lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",-79.194353,43.806686,2,Fast Food Restaurant,Women's Store,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",-79.160497,43.784535,4,Bar,Women's Store,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",-79.188711,43.763573,3,Electronics Store,Spa,Rental Car Location,Mexican Restaurant,Medical Center,Breakfast Spot,Pizza Place,Garden Center,Creperie,Diner
3,M1G,Scarborough,Woburn,-79.216917,43.770992,1,Coffee Shop,Korean Restaurant,Insurance Office,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
4,M1H,Scarborough,Cedarbrae,-79.239476,43.773136,3,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Athletics & Sports,Hakka Restaurant,Drugstore,Dog Run,Discount Store
5,M1J,Scarborough,Scarborough Village,-79.239476,43.744734,3,Playground,Convenience Store,Cupcake Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",-79.262029,43.727929,3,Discount Store,Chinese Restaurant,Department Store,Convenience Store,Coffee Shop,Bus Station,Drugstore,Dog Run,Dumpling Restaurant,Curling Ice
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",-79.284577,43.711112,3,Bakery,Bus Line,Bus Station,Fast Food Restaurant,Metro Station,Intersection,Soccer Field,Park,Dog Run,Discount Store
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",-79.239476,43.716316,3,Motel,American Restaurant,Women's Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
9,M1N,Scarborough,"Birch Cliff, Cliffside West",-79.264848,43.692657,3,Skating Rink,College Stadium,General Entertainment,Café,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Women's Store


In [110]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude+.024, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torronto_merged['Lat'], torronto_merged['Lon'], torronto_merged['Neighborhood'], torronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [78]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 0, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Women's Store,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
2,Scarborough,0,Electronics Store,Spa,Rental Car Location,Mexican Restaurant,Medical Center,Breakfast Spot,Pizza Place,Garden Center,Creperie,Diner
4,Scarborough,0,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Bank,Athletics & Sports,Hakka Restaurant,Drugstore,Dog Run,Discount Store
5,Scarborough,0,Playground,Convenience Store,Cupcake Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
7,Scarborough,0,Bakery,Bus Line,Bus Station,Fast Food Restaurant,Metro Station,Intersection,Soccer Field,Park,Dog Run,Discount Store
8,Scarborough,0,Motel,American Restaurant,Women's Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
10,Scarborough,0,Indian Restaurant,Latin American Restaurant,Vietnamese Restaurant,Chinese Restaurant,Light Rail Station,Pet Store,Gay Bar,Cuban Restaurant,Dumpling Restaurant,Drugstore
11,Scarborough,0,Middle Eastern Restaurant,Breakfast Spot,Shopping Mall,Bakery,Sandwich Place,Auto Garage,Dance Studio,Dumpling Restaurant,Drugstore,Dog Run
12,Scarborough,0,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
13,Scarborough,0,Pizza Place,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Fried Chicken Joint,Fast Food Restaurant,Noodle House,Pharmacy,Dessert Shop,Dim Sum Restaurant


### Cluster 2

In [79]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 1, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1,Playground,Park,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
23,North York,1,Park,Bar,Bank,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
25,North York,1,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
30,North York,1,Bus Stop,Airport,Park,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Dog Run
40,East York,1,Coffee Shop,Park,Convenience Store,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
44,Central Toronto,1,Bus Line,Park,Swim School,Women's Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
50,Downtown Toronto,1,Park,Trail,Playground,Women's Store,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
64,Central Toronto,1,Park,Sushi Restaurant,Jewelry Store,Bus Line,Trail,Women's Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store
74,York,1,Park,Women's Store,Market,Fast Food Restaurant,Pharmacy,Drugstore,Dumpling Restaurant,Dog Run,Cupcake Shop,Discount Store
79,North York,1,Construction & Landscaping,Park,Basketball Court,Bakery,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Women's Store
